# 2.3 Creating a PoS-tagger from data

## Question 1: Create your own tagger
For this exercise, you should recall the resources you've been studying for task 2.1 and then read the following web-page to understand the exact specification of an nltk tagger: https://www.nltk.org/book/ch05.html.

You can observe that an nltk tagger takes the form of an object in python for which different methods can be called. You should in particular look at the `nltk.UnigramTagger`:
```python
my_unigram_tagger = nltk.UnigramTagger(...)
```
As can be seen in the examples above, `nltk.UnigramTagger` can be instantiated by passing as argument either a tagged text in the same format as that of `corpus.tagged_sents()` or with the attribute `model=likely_tags`, where `likely_tags` is a dictionnary where the keys are the words and the values are the most likely tag (the associated tag with highest count in the training text).

In this task, you are asked to build a tagger based on a unigram tagger. If care is not taken, many words could be assigned a tag of `None`, because they were not among the words seen during training. How can you solve this to improve performance? We would like to assign a default tag in those cases. In other words, we want to use the unigram tagger first, and if it is unable to assign a tag, then use the default tagger. This process is known as backoff in the nltk package (not to be confused with the concept of backoff smoothing for N-grams). For that, the backoff parameter that can be used when creating the tagger will be key as shown below where it is assumed that the default tag will always be a noun.
```python
baseline_tagger = nltk.UnigramTagger(model=likely_tags, backoff=nltk.DefaultTagger('NN'))
```
In our case, you should build a smarter backoff such that it reverts to the most frequent tag associated to the first occurences of each word in the text. In other words, suppose that you pre-process your text by replacing the first occurrence of any word with the `<UNK>` token, the backoff will be the most frequent tag associated to `<UNK>`.

In [12]:
import nltk
from collections import defaultdict, Counter

def build_tagger(corpus):
    """
    Returns a unigram nltk tagger trained on the given corpus with
    the specificity that the backoff should be obtained as the most
    frequent tag associated to <UNK> where <UNK> is associated to
    the first occurrence of any word in the text.

    Parameters
    -----------
    corpus: a `TaggedCorpusReader` object created from some
        text representation of the corpus (ie BAWE_train)

    Returns
    -------
    Returns an nltk Tagger which assigns the most likely tag to any
    word from the corpus and falls back to a DefaultTagger which
    returns the most likely tag for the <UNK> token.

    e.g. this object can be obtained by nltk.UnigramTagger(text)

    Warning
    -------
    Pay attention to the fact that when training your tagger, the
    first occurence of any word should be replaced by the <UNK> token.
    """
    tagged_words_corpus = corpus.tagged_words()
    seen = set()
    twc = [x for x in tagged_words_corpus]
    for i, (w, tag) in enumerate(tagged_words_corpus):
        if w not in seen:
            twc[i] = ('<UNK>', tag)
            seen.add(w)
    
    cnt = defaultdict(int)
    for x, y in twc:
        if x == '<UNK>':
            cnt[y] += 1
    c = Counter(cnt)
    default_tag = c.most_common(1)[0][0]
    
    tsc = [x for x in corpus.tagged_sents()]
    seen = set()
    for i, sent in enumerate(corpus.tagged_sents()):
        for j, (w, tag) in enumerate(sent):
            if w not in seen:
                tsc[i][j] = ('<UNK>', tag)
                seen.add(w)
        
    tagger = nltk.UnigramTagger(tsc, backoff=nltk.DefaultTagger(default_tag))
    
    return tagger

## Question 2: Testing your tagger
An nltk tagger has a built-in method tag that you can use as follows:
```python
sentence = "This is an example sentence"
my_tagger.tag(sentence.split())
```
As shown in the example, the `tag` method accepts an input which is an untagged input text split into a list of strings. This method will return a list of tuples where each tuple consists of a word (coming from the input) and its associated tag.

To check this, you are asked to test your tagger on the following sentence `"Can you tag this simple sentence with the tagger you just built ?"` and provide the result below (in the format provided by the method `tag`).

In [13]:
from nltk.corpus.reader import TaggedCorpusReader

train = TaggedCorpusReader(root="resources", fileids="BAWE_train.retagged.txt")

sentence = "Can you tag this simple sentence with the tagger you just built ?"
my_tagger = build_tagger(train)
print(my_tagger.tag(sentence.split()))

[('Can', 'NN'), ('you', 'PRP'), ('tag', 'NN'), ('this', 'DT'), ('simple', 'JJ'), ('sentence', 'NN'), ('with', 'IN'), ('the', 'DT'), ('tagger', 'NN'), ('you', 'PRP'), ('just', 'RB'), ('built', 'VBN'), ('?', '.')]


# Question 3: Evaluate tagger
The previous question showed you how to obtain the tags associated to words in a text. In general, you'll be interested to evaluate how good you tagger is.

An nltk tagger offers an `evaluate` method which you can use as shown below:
```python
my_tagger.evaluate(expression)
```
where `expression` is a list of tagged sentences (in the same format as that of `corpus.tagged_sents()`. In other words, a list of tagged sentences each broken down to a list of `(word, tag)` tuples). The `evaluate` method returns a float that represents the accuracy of the tagger. That is, the proportion of tags that were assigned correctly by the tagger on the tested set of tagged sentences.

Below, we ask you to report the test error rate of your tagger (in %). This corresponds to 100%−accuracy. The answer you provide should be given in the form of a percentage (float number, without the % sign). So for instance, if your tagger has a "20.2%" error rate, you should report `20.2`. As usual, you should report the error rate with at least two significant digits.

In [15]:
test = TaggedCorpusReader(root="resources", fileids="BAWE_test.retagged.txt")

print(100 * (1 - my_tagger.evaluate(test.tagged_sents())))

12.641742845096394
